In [22]:
from bs4 import BeautifulSoup
import urllib.request as req
import os.path
import re
import pandas as pd
import numpy as np
import urllib
import sys

In [23]:
sys.version

'3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]'

In [58]:
main_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=181381&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='

In [25]:
res = req.urlopen(main_url)

[파이썬 3.7대에 html.parser 사용시 예외 발생 --> lxml 사용]

In [27]:
soup = BeautifulSoup(res, 'lxml')

In [30]:
soup.select("span#_filtered_ment_2")[0]

<span id="_filtered_ment_2">
														
															
															
																배우라면 감독이라면 제발 관수로 죠지지말고 연기랑 스토리로 죠져라 부탁한다 
															
														
														
													</span>

[리뷰데이터 추출]
- class = score_reple 내에 id = _filtered_ment_ + n 에 리뷰가 있다
- 하지만, 리뷰가 긴 경우 글이 folded 된 상태이며, 위 에서 또 하위 태그로 들어가 a태그 내에 존재함

1) review folded

In [12]:
soup.select('.score_reple')[1].find('span', id = '_filtered_ment_'+str(1))

<span id="_filtered_ment_1">
<span class="_unfold_ment" id="_unfold_ment1">
<a data-src="천문, 천만될 가능성이 높습니다.다만 백두산에 상영관 밀리는 게 아쉽네요.정말 재미있고 탄탄한 스토리로 몰입해서 봤습니다.세종대왕이 생존해 계시다면정말 한석규님과 똑같이 생겼을 것 같네요..뿌리깊은나무, 천문까지..세종대왕 역할 찰떡입니다..추천해주신 분들은 2020년도 행복하길♡ " href="javascript:void(0);" onclick="unfoldPointMent(this);">
																		천문, 천만될 가능성이 높습니다.다만 백두산에 상영관 밀리는 게 아쉽네요.정말 재미있고 탄탄한 스토리로 몰입해서 봤습니다.세종대왕이 생존해 계시다면정말 한석규님과 똑같이 생겼을 것 같네요..뿌리깊은나무, 천문까지.....
																	</a>
</span>
</span>

In [13]:
soup.select('.score_reple')[1].find('span', id = '_filtered_ment_'+str(1)).find('a')['data-src']

'천문, 천만될 가능성이 높습니다.다만 백두산에 상영관 밀리는 게 아쉽네요.정말 재미있고 탄탄한 스토리로 몰입해서 봤습니다.세종대왕이 생존해 계시다면정말 한석규님과 똑같이 생겼을 것 같네요..뿌리깊은나무, 천문까지..세종대왕 역할 찰떡입니다..추천해주신 분들은 2020년도 행복하길♡ '

2) unfolded 

In [14]:
soup.select('.score_reple')[2].find('span', id = '_filtered_ment_'+str(2)).string

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t최민식과 한석규라를 두 배우만으로도 볼 이유가 충분했던 영화지만, 역사 책에서 볼 수 업었던 상상력을 충분히 정성스럽게 표현했다! \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t'

### 전체 데이터 수집

[페이지 수 계산]

In [31]:
soup.select('.total')[0].text

'관람객 평점 12,463건내 평점 등록'

In [33]:
p_num = re.compile("[0-9]+")

In [34]:
review_num = int(p_num.findall(soup.select('.total')[0].text)[0]+p_num.findall(soup.select('.total')[0].text)[1])

In [35]:
page_num = int(np.ceil(review_num/10))

In [59]:
reviews = []
ids = []
dts = []
star_score = []
sym_num = []
p_num = re.compile('[0-9]+')

for page in range(1, page_num+1):
    url_page = main_url+str(page)
    res1 = req.urlopen(url_page)
    soup1 = BeautifulSoup(res1, 'lxml')
    
    i = 0

    while True:
        try:
            a = soup1.select('.score_reple')[i].find('span', id = '_filtered_ment_'+str(i))
            if len(a) > 1:
                reviews.append(a.find('a')['data-src'])
            else:
                reviews.append(a.get_text())
                
            # id 수집
            ids.append(soup1.select('dt span')[i].text)
            
            # date 수집
            dts.append(soup1.findAll('dt')[i].select('em')[1].text)
            
            # scroe 수집
            star_score.append(p_num.findall(soup1.select('.star_score')[i].get_text()))
            
            # 공감/비공감 수집
            sym_num.append(p_num.findall(soup1.select('.btn_area')[i].get_text()))
            i = i+1
        except IndexError as e:
            print('%d page 작업완료' % page)
            break

1 page 작업완료
2 page 작업완료
3 page 작업완료
4 page 작업완료
5 page 작업완료
6 page 작업완료
7 page 작업완료
8 page 작업완료
9 page 작업완료
10 page 작업완료
11 page 작업완료
12 page 작업완료
13 page 작업완료
14 page 작업완료
15 page 작업완료
16 page 작업완료
17 page 작업완료
18 page 작업완료
19 page 작업완료
20 page 작업완료
21 page 작업완료
22 page 작업완료
23 page 작업완료
24 page 작업완료
25 page 작업완료
26 page 작업완료
27 page 작업완료
28 page 작업완료
29 page 작업완료
30 page 작업완료
31 page 작업완료
32 page 작업완료
33 page 작업완료
34 page 작업완료
35 page 작업완료
36 page 작업완료
37 page 작업완료
38 page 작업완료
39 page 작업완료
40 page 작업완료
41 page 작업완료
42 page 작업완료
43 page 작업완료
44 page 작업완료
45 page 작업완료
46 page 작업완료
47 page 작업완료
48 page 작업완료
49 page 작업완료
50 page 작업완료
51 page 작업완료
52 page 작업완료
53 page 작업완료
54 page 작업완료
55 page 작업완료
56 page 작업완료
57 page 작업완료
58 page 작업완료
59 page 작업완료
60 page 작업완료
61 page 작업완료
62 page 작업완료
63 page 작업완료
64 page 작업완료
65 page 작업완료
66 page 작업완료
67 page 작업완료
68 page 작업완료
69 page 작업완료
70 page 작업완료
71 page 작업완료
72 page 작업완료
73 page 작업완료
74 page 작업완료
75 page 작업완료
76 page 작업완료
77 page 작업완료
78 page 

595 page 작업완료
596 page 작업완료
597 page 작업완료
598 page 작업완료
599 page 작업완료
600 page 작업완료
601 page 작업완료
602 page 작업완료
603 page 작업완료
604 page 작업완료
605 page 작업완료
606 page 작업완료
607 page 작업완료
608 page 작업완료
609 page 작업완료
610 page 작업완료
611 page 작업완료
612 page 작업완료
613 page 작업완료
614 page 작업완료
615 page 작업완료
616 page 작업완료
617 page 작업완료
618 page 작업완료
619 page 작업완료
620 page 작업완료
621 page 작업완료
622 page 작업완료
623 page 작업완료
624 page 작업완료
625 page 작업완료
626 page 작업완료
627 page 작업완료
628 page 작업완료
629 page 작업완료
630 page 작업완료
631 page 작업완료
632 page 작업완료
633 page 작업완료
634 page 작업완료
635 page 작업완료
636 page 작업완료
637 page 작업완료
638 page 작업완료
639 page 작업완료
640 page 작업완료
641 page 작업완료
642 page 작업완료
643 page 작업완료
644 page 작업완료
645 page 작업완료
646 page 작업완료
647 page 작업완료
648 page 작업완료
649 page 작업완료
650 page 작업완료
651 page 작업완료
652 page 작업완료
653 page 작업완료
654 page 작업완료
655 page 작업완료
656 page 작업완료
657 page 작업완료
658 page 작업완료
659 page 작업완료
660 page 작업완료
661 page 작업완료
662 page 작업완료
663 page 작업완료
664 page 작업완료
665 page 작업완료
666 pa

1170 page 작업완료
1171 page 작업완료
1172 page 작업완료
1173 page 작업완료
1174 page 작업완료
1175 page 작업완료
1176 page 작업완료
1177 page 작업완료
1178 page 작업완료
1179 page 작업완료
1180 page 작업완료
1181 page 작업완료
1182 page 작업완료
1183 page 작업완료
1184 page 작업완료
1185 page 작업완료
1186 page 작업완료
1187 page 작업완료
1188 page 작업완료
1189 page 작업완료
1190 page 작업완료
1191 page 작업완료
1192 page 작업완료
1193 page 작업완료
1194 page 작업완료
1195 page 작업완료
1196 page 작업완료
1197 page 작업완료
1198 page 작업완료
1199 page 작업완료
1200 page 작업완료
1201 page 작업완료
1202 page 작업완료
1203 page 작업완료
1204 page 작업완료
1205 page 작업완료
1206 page 작업완료
1207 page 작업완료
1208 page 작업완료
1209 page 작업완료
1210 page 작업완료
1211 page 작업완료
1212 page 작업완료
1213 page 작업완료
1214 page 작업완료
1215 page 작업완료
1216 page 작업완료
1217 page 작업완료
1218 page 작업완료
1219 page 작업완료
1220 page 작업완료
1221 page 작업완료
1222 page 작업완료
1223 page 작업완료
1224 page 작업완료
1225 page 작업완료
1226 page 작업완료
1227 page 작업완료
1228 page 작업완료
1229 page 작업완료
1230 page 작업완료
1231 page 작업완료
1232 page 작업완료
1233 page 작업완료
1234 page 작업완료
1235 page 작업완료
1236 page 

In [60]:
sym_num

[['3390', '265'],
 ['2714', '400'],
 ['2350', '264'],
 ['1889', '340'],
 ['1450', '166'],
 ['1397', '135'],
 ['1344', '125'],
 ['1211', '102'],
 ['1196', '112'],
 ['1201', '137'],
 ['910', '124'],
 ['760', '58'],
 ['788', '109'],
 ['732', '95'],
 ['670', '56'],
 ['611', '88'],
 ['616', '108'],
 ['533', '53'],
 ['503', '30'],
 ['526', '63'],
 ['522', '71'],
 ['456', '34'],
 ['486', '82'],
 ['398', '35'],
 ['420', '65'],
 ['382', '36'],
 ['379', '58'],
 ['373', '56'],
 ['384', '85'],
 ['350', '62'],
 ['290', '35'],
 ['283', '48'],
 ['284', '52'],
 ['258', '33'],
 ['245', '27'],
 ['214', '12'],
 ['227', '25'],
 ['233', '31'],
 ['236', '34'],
 ['231', '30'],
 ['230', '30'],
 ['201', '9'],
 ['214', '24'],
 ['236', '47'],
 ['219', '32'],
 ['188', '11'],
 ['196', '26'],
 ['176', '20'],
 ['179', '26'],
 ['175', '28'],
 ['265', '119'],
 ['165', '24'],
 ['145', '7'],
 ['143', '21'],
 ['146', '27'],
 ['122', '8'],
 ['120', '11'],
 ['111', '11'],
 ['116', '16'],
 ['97', '8'],
 ['131', '42'],
 ['97

review 데이터 정제(웹과 동일하게)

In [61]:
reviews

['\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t한석규의 세종은 어떻게 해야 저런 연기가 가능한지 의문을 남긴다. 세종대왕 연기의 끝을 보여줌. \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t',
 '천문, 천만될 가능성이 높습니다.다만 백두산에 상영관 밀리는 게 아쉽네요.정말 재미있고 탄탄한 스토리로 몰입해서 봤습니다.세종대왕이 생존해 계시다면정말 한석규님과 똑같이 생겼을 것 같네요..뿌리깊은나무, 천문까지..세종대왕 역할 찰떡입니다..추천해주신 분들은 2020년도 행복하길♡ ',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t최민식과 한석규라를 두 배우만으로도 볼 이유가 충분했던 영화지만, 역사 책에서 볼 수 업었던 상상력을 충분히 정성스럽게 표현했다! \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t백두산에서 온 알바들아~~평점테러 하지마라!! 백두산에 비하면

In [51]:
reviews_final = [''.join(re.findall('[^\t\r\n]',review)) for review in reviews]

In [52]:
reviews_final[:3]

['약간 지루해서 내용이 기억이 잘 안나요.내용이 영화에서 본건지 꿈에서 본건지 애들 책에서 본건지..아쉽다 ',
 '세종대왕과 장영실이라는 훌륭한 소재로 이렇게 재미없게 영화를 만들 수 있다니 놀랍습니다. 다만 안여와 혼천의는 잘 구현해낸 것 같군요. ',
 '어색한 브로맨스 질질끄는 지루함 ']

In [53]:
review_df = pd.DataFrame(reviews_final, columns = ['review'])

In [54]:
# id
ids_df = pd.DataFrame(ids, columns=['id'])
# date
dts_df = pd.DataFrame(dts, columns=['date'])
# score
score_df = pd.DataFrame(star_score,columns=['score'])
# like/dislike
sym_df = pd.DataFrame(sym_num, columns=['like','dislike'])

In [55]:
# 데이터프레임 합치기
raw_temp = review_df.join(ids_df, how='left')
raw_temp2 = raw_temp.join(dts_df, how='left')
raw_temp3 = raw_temp2.join(score_df, how='left')
raw_data = raw_temp3.join(sym_df, how= 'left')

In [56]:
raw_data

,review,id,date,score,like,dislike
0,약간 지루해서 내용이 기억이 잘 안나요.내용이 영화에서 본건지 꿈에서 본건지 애들 ...,Jess(giss****),2020.03.05 07:39,6,12,67
1,세종대왕과 장영실이라는 훌륭한 소재로 이렇게 재미없게 영화를 만들 수 있다니 놀랍습...,maxkim(maxk****),2020.02.29 13:27,4,25,141
2,어색한 브로맨스 질질끄는 지루함,Yim(kawa****),2020.02.28 20:33,2,33,171
3,약간 지루해서 내용이 기억이 잘 안나요.내용이 영화에서 본건지 꿈에서 본건지 애들 ...,Jess(giss****),2020.03.05 07:39,6,12,67
4,세종대왕과 장영실이라는 훌륭한 소재로 이렇게 재미없게 영화를 만들 수 있다니 놀랍습...,maxkim(maxk****),2020.02.29 13:27,4,25,141
...,...,...,...,...,...,...
3736,세종대왕과 장영실이라는 훌륭한 소재로 이렇게 재미없게 영화를 만들 수 있다니 놀랍습...,maxkim(maxk****),2020.02.29 13:27,4,25,141
3737,어색한 브로맨스 질질끄는 지루함,Yim(kawa****),2020.02.28 20:33,2,33,171
3738,약간 지루해서 내용이 기억이 잘 안나요.내용이 영화에서 본건지 꿈에서 본건지 애들 ...,Jess(giss****),2020.03.05 07:39,6,12,67
3739,세종대왕과 장영실이라는 훌륭한 소재로 이렇게 재미없게 영화를 만들 수 있다니 놀랍습...,maxkim(maxk****),2020.02.29 13:27,4,25,141


In [44]:
raw_data.to_csv('chunmoon_0611.csv', index = False)

In [49]:
raw_data.loc[2172]['review']

'약간 지루해서 내용이 기억이 잘 안나요.내용이 영화에서 본건지 꿈에서 본건지 애들 책에서 본건지..아쉽다 '